In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import sys
from logging import INFO, StreamHandler, getLogger

logger = getLogger()
if not logger.hasHandlers():
    logger.addHandler(StreamHandler(sys.stdout))
logger.setLevel(INFO)

# Import

In [ ]:
import copy
import os
import pathlib

import matplotlib.pyplot as plt
import numpy as np
from src.config import Kohyama21ModelConfig
from src.model import Kohyama21Model
from src.random_seed_helper import set_all_seeds

plt.rcParams["font.family"] = "serif"

# Define constants

In [ ]:
ROOT_DIR = str((pathlib.Path(os.environ["PYTHONPATH"]) / "..").resolve())

# Define methods

In [ ]:
def calculate_lag_corr(
    x1: np.ndarray, x2: np.ndarray, nlag: int
) -> tuple[np.ndarray, np.ndarray]:
    #
    assert x1.shape == x2.shape
    assert x1.ndim == 1
    assert nlag > 0.0 and isinstance(nlag, int)

    lags = np.arange(-nlag, nlag + 1, 1)
    ntime = len(x1)
    cors = np.zeros((2 * nlag + 1))

    for ilag in range(0, nlag + 1):
        x1_tmp = x1[0 : ntime - ilag]
        x2_tmp = x2[ilag:ntime]
        assert len(x1_tmp) == len(x2_tmp)
        cors[nlag - ilag] = np.corrcoef(x1_tmp, x2_tmp)[0, 1]

        if ilag == 0:
            continue

        x1_tmp = x1[ilag:ntime]
        x2_tmp = x2[0 : ntime - ilag]
        assert len(x1_tmp) == len(x2_tmp)
        cors[nlag + ilag] = np.corrcoef(x1_tmp, x2_tmp)[0, 1]

    return lags, cors

# Make config

In [ ]:
# Configuration used in Kohyama et al. (2021).
# https://www.science.org/doi/full/10.1126/science.abh3295

config = Kohyama21ModelConfig(
    tmax_year=1000.0,
    dt_year=1.0 / 24.0,
    istep_out=2,
    ny=101,
    Lx_1=8.25e6,
    Lx_2=4.95e6,
    H_1=300.0,
    H_2=300.0,
    R_1=5.0e4,
    R_2=5.0e4,
    delta_1=1.0e5,
    delta_2=1.0e5,
    Lx=2.75e7,
    Ly=1.0e7,
    D_a=1e4,
    lamd=50.0,
    rho_a=1.25,
    rho_w=1e3,
    C_pa=1e3,
    C_pw=4e3,
    nu_a=5e-7,
    eps=1e3,
    B=2.5,
    F=125.0,
    S_a=5e-3,
    f_0=1e-4,
    beta=2e-11,
    d_l=5822.785,
    amp_noise=5.0,
    amp_noise_theta_1=9.0,
    amp_noise_theta_2=9.0,
)

config.save("KO21.yml")
_config = Kohyama21ModelConfig.load("KO21.yml")
assert config.__dict__ == _config.__dict__

In [ ]:
# Configuration used in Gallego and Cessi (2001)
# https://doi.org/10.1175/1520-0442(2001)014%3C2815:DVOTOA%3E2.0.CO;2

config = Kohyama21ModelConfig(
    tmax_year=1000.0,
    dt_year=1.0 / 24.0,
    istep_out=2,
    ny=101,
    Lx_1=8.25e6,
    Lx_2=4.95e6,
    H_1=1000.0,
    H_2=1000.0,
    R_1=2.8e4,
    R_2=2.8e4,
    delta_1=5.2e3,
    delta_2=5.2e3,
    Lx=2.75e7,
    Ly=1.0e7,
    D_a=1e4,
    lamd=50.0,
    rho_a=1.25,
    rho_w=1e3,
    C_pa=1e3,
    C_pw=4e3,
    nu_a=5e-7,
    eps=1e3,
    B=2.5,
    F=125.0,
    S_a=5e-3,
    f_0=1e-4,
    beta=2e-11,
    d_l=5822.785,
    amp_noise=0.0,
    amp_noise_theta_1=0.0,
    amp_noise_theta_2=0.0,
)

config.save("GC01.yml")
_config = Kohyama21ModelConfig.load("GC01.yml")
assert config.__dict__ == _config.__dict__

# Run model

In [ ]:
# set_all_seeds(42)
# experiment_name = "KO21"
# config = Kohyama21ModelConfig.load(f"{experiment_name}.yml")
# model = Kohyama21Model(config)

In [ ]:
set_all_seeds(42)
experiment_name = "GC01"
config = Kohyama21ModelConfig.load(f"{experiment_name}.yml")
model = Kohyama21Model(config)

In [ ]:
T1_out, T2_out = model.run_simulation()

spinup_period = 12 * 20
T1_out = T1_out[spinup_period:]
T2_out = T2_out[spinup_period:]

# Plot results

In [ ]:
ts = np.linspace(0.0, config.tmax_year, (config.nt // config.istep_out) + 1)
ts = ts[spinup_period:]
ys = copy.deepcopy(model.y) / 1e3
ts, ys = np.meshgrid(ts, ys, indexing="ij")

vmin, vmax = np.quantile(T1_out.flatten(), 0.01), np.quantile(T1_out.flatten(), 0.99)

plt.rcParams["font.size"] = 18
fig, axes = plt.subplots(1, 2, sharex=True, sharey=True, figsize=[12, 5])

for ax, data, ttl in zip(
    axes, [T1_out, T2_out], [r"$T_{\rm Pacific}$", r"$T_{\rm Atlantic}$"]
):
    cnt = ax.pcolormesh(ts, ys, data, vmin=vmin, vmax=vmax)
    fig.colorbar(cnt, ax=ax)
    ax.set_ylabel("North-South [km]")
    ax.set_xlabel("Time [yr]")
    ax.set_title(ttl)
plt.suptitle(f"{experiment_name=}")
plt.tight_layout()
plt.show()

In [ ]:
ts = np.linspace(0.0, config.tmax_year, (config.nt // config.istep_out) + 1)
ts = ts[spinup_period:]
ys = copy.deepcopy(model.y) / 1e3

vmin, vmax = np.quantile(T1_out.flatten(), 0.01), np.quantile(T1_out.flatten(), 0.99)
sy, ey = 40, 60

plt.rcParams["font.size"] = 16
fig, axes = plt.subplots(1, 2, sharex=True, sharey=True, figsize=[12, 5])

for ax, data, ttl, c in zip(
    axes,
    [T1_out, T2_out],
    [r"$T_{\rm Pacific}$", r"$T_{\rm Atlantic}$"],
    ["coral", "deepskyblue"],
):
    ave = np.mean(data[:, sy:ey], axis=1)
    ax.plot(ts, ave, color=c)
    ax.set_ylabel(ttl)
    ax.set_xlabel("Time [yr]")
    ax.set_title(f"{ttl}\n(average between y = {int(ys[sy])} ~ {int(ys[ey])} km)")
plt.suptitle(f"{experiment_name=}")
plt.tight_layout()
plt.show()

In [ ]:
sy, ey = 40, 60
t1 = np.mean(T1_out[:, sy:ey], axis=1)
t2 = np.mean(T2_out[:, sy:ey], axis=1)
lags, cors = calculate_lag_corr(t1, t2, 12 * 20)

lags = lags / 12  # to year

plt.rcParams["font.size"] = 15
fig = plt.figure()
plt.plot(lags, cors)
plt.xlabel("Lag [yr]")
plt.ylabel("Corr.")
plt.axvline(0, ls="--", color="k")
plt.axhline(0, ls="--", color="k")
plt.suptitle(f"{experiment_name=}")
plt.tight_layout()
plt.show()